# 01 Build Macro Monthly Panel

Align mixed-frequency series to a month-end panel.


## Table of Contents
- Load series
- Month-end alignment
- Missingness
- Save processed panel


## Why This Notebook Matters
Data notebooks build the datasets used everywhere else. If these steps are wrong, every model result is suspect.
You will practice:
- API ingestion and caching,
- frequency alignment,
- label construction.


## What You Will Produce
- data/processed/panel_monthly.csv

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.
- You can point to the concrete deliverable(s) listed below and explain how they were produced.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Merging mixed-frequency series without explicit resampling/aggregation.
- Forgetting to shift targets for forecasting tasks.

## Matching Guide
- `docs/guides/01_data/01_build_macro_monthly_panel.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/01_data/01_build_macro_monthly_panel.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Build a clean month-end panel of predictors (mixed daily/monthly series).


## Your Turn: Load Raw Series


In [ ]:
import pandas as pd
from src import data as data_utils

# TODO: Load cached JSON series into individual DataFrames
# Convert to numeric and set DatetimeIndex
...


## Your Turn: Align to Month-End


In [ ]:
# TODO: resample to month-end (ME) and forward-fill
# Save to data/processed/panel_monthly.csv
...


## Checkpoint


In [ ]:
# TODO: Assert no missing values after forward-fill (or explain remaining NaNs)
...


## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# Expected file: data/processed/panel_monthly.csv
# TODO: After saving your processed dataset, load it and run checks.
# df = pd.read_csv(PROCESSED_DIR / 'your_file.csv', index_col=0, parse_dates=True)
# assert df.index.is_monotonic_increasing
# assert df.shape[0] > 20
# print(df.dtypes)
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Load series</summary>

```python
import pandas as pd
from src import data as data_utils
from src import fred_api

raw_dir = RAW_DIR / 'fred'
series_ids = ['UNRATE', 'FEDFUNDS', 'CPIAUCSL', 'INDPRO', 'RSAFS', 'T10Y2Y']

frames = []
for sid in series_ids:
    payload = data_utils.load_json(raw_dir / f'{sid}.json') if (raw_dir / f'{sid}.json').exists() else None
    if payload is None:
        # Offline fallback
        panel = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)
        break
    frames.append(fred_api.observations_to_frame(payload, sid))
else:
    panel = pd.concat(frames, axis=1).sort_index()

panel.head()
```

</details>

<details><summary>Solution: Month-end alignment</summary>

```python
# Convert to month-end panel (ME) and forward-fill.
panel_monthly = panel.resample('ME').last().ffill()
panel_monthly.tail()
```

</details>

<details><summary>Solution: Missingness</summary>

```python
missing = panel_monthly.isna().sum().sort_values(ascending=False)
missing.head(10)
```

</details>

<details><summary>Solution: Save processed panel</summary>

```python
from src import data as data_utils
data_utils.save_csv(panel_monthly, PROCESSED_DIR / 'panel_monthly.csv')
print('saved', PROCESSED_DIR / 'panel_monthly.csv')
```

</details>

